In [4]:
# Importing necessary libraries
import pandas as pd               
import numpy as np                
import matplotlib.pyplot as plt   
import os                         
import joblib                     
import hopsworks                  

import sys
sys.path.append('Feature_pipeline.ipynb')  # Replace with the actual path
from feature_pipeline import amd_fg   #Loading in the amd_fg
#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

ModuleNotFoundError: No module named 'feature_pipeline'